In [1]:
#required packages
import os
import pandas as pd
import hmac, hashlib;
import time;
import base64;
import requests;
import ntplib
import numpy as np
from dotenv import load_dotenv
import geopy
from convertbng.util import convert_bng, convert_lonlat
from scipy.spatial import cKDTree

In [2]:
#custom ETL scripts
from etl_functions import tascomi_utils
from etl_functions import exacom_utils

In [3]:
pd.options.display.max_colwidth = 1000

## Setup

In [4]:
#Set up .env variables
load_dotenv()

True

In [5]:
#Initialise geolocator
geolocator = geopy.geocoders.Nominatim(user_agent="buildup_wf")

In [6]:
#Dictionary for matching planning and BC applications
matches_dict = {
    'planning_id': [],
    'bc_dist': [],
    'bc_id': []
}


## Scheme-level data

In [11]:
## PIPELINE OUTLINE

##1. EXTRACT PLANNING DATA

#Take accommodation schedule with PP reference - example doesn't have one, so we use a known planning ref
planning_ref = '202512'

#Use PP reference to get matching planning application from Tascomi Planning
planning_data = (
    tascomi_utils.get_data('planning', 'applications', optional_param='application_reference_number', optional_param_value=planning_ref)
    .pipe(tascomi_utils.locate_apps, database='planning')
    .pipe(tascomi_utils.clean_dates, dates=['decision_issued_date'])
    .apply(lambda row: tascomi_utils.geocode_address(row) if not row.x_coordinate else row, axis=1)
)


In [12]:
planning_data

,planning_id,planning_portal_reference,application_reference_number,decision_type_id,site_address_description,dtf_location_id,proposal,received_date,decision_issued_date,id_y,uprn,usrn,lpi_key,pao_text,postcode,x_coordinate,y_coordinate,blpu_logical_status_code,lpi_logical_status_code
0,45979,None,202512,2,"Homebase,2c,Fulbourne Road,Walthamstow,London,E17 4EE",1250679,"&lt;span style=&quot;font-size:11.0pt;font-family:&amp;quot;Arial&amp;quot;,sans-serif;\r\nmso-fareast-font-family:Calibri;mso-fareast-theme-font:minor-latin;mso-ansi-language:\r\nEN-GB;mso-fareast-language:EN-US;mso-bidi-language:AR-SA&quot;&gt;Demolition and\r\nredevelopment of the site to provide a mixed use scheme comprising up to 583\r\nresidential units (Class C3 and including Private Rented Sector) in 8 buildings\r\nranging from 4 to 18 storeys, commercial uses (flexible retail, community and\r\nleisure uses), new access from Fulbourne Road, car parking; provision of new\r\nplant and renewable energy equipment; creation of servicing areas and provision\r\nof associated services, including waste, refuse, cycle storage, and lighting;\r\nand new routes and open spaces within the development and associated works and\r\noperations. (Amended description - Further information/amended Environmental\r\nImpact Assessment).&lt;/span&gt;",2020-08-19 00:00:00,2021-04-20,1250679,100022980796,22838500,5930L000044821,HOMEBASE,E17 4EE,538468,190114,1,1


In [13]:
##2. EXTRACT BC DATA

#load in BC data - daily refresh
BC_data = (
    tascomi_utils.get_data('build', 'applications', optional_param='received_date', optional_param_value='2021-11-19')
    .pipe(tascomi_utils.locate_apps, database='build')
    .pipe(tascomi_utils.clean_dates, dates=['received_date', 'valid_date', 'decision_date', 'started_date', 'completed_date'])
    .pipe(tascomi_utils.filter_bc)
)

In [14]:
BC_data

,build_id,application_number,description,dtf_location_id,received_date,valid_date,decision_date,started_date,completed_date,site_description,id_y,uprn,usrn,lpi_key,pao_text,postcode,x_coordinate,y_coordinate,blpu_logical_status_code,lpi_logical_status_code
0,3311792,2863/21/IN,Demolition of existing single storey structures and construct two new dwelllings.,4155061,2021-11-19,NaT,2021-11-23,NaT,NaT,,4155061,100023013609,22879450,5930L000048324,,E7 9EL,539575,185820,1,1
1,3311793,2864/21/PA/MAST,"New mixed use development of 583 residential units and 3 commercial spaces. Comprising of 6 Blocks (Block A, 173 Units, Block B, 24 Units, Block C, 156 Unit, Block D , 48 Units, Block E, 73 Units \r\nBlock F, 109 Units) and 1 Hub, collectively known as ""Patchworks""\r\n",4227964,2021-11-19,2021-11-19,2022-01-12,2022-01-13,NaT,"6 Blocks, and 1 Hub collectively known as ""Patchworks""\r\nThe Hub, \r\nBlock A – 173 Units, \r\nBlock B – 24 Units, \r\nBlock C – 156 Unit \r\nBlock D – 48 Units, \r\nBlock E – 73 Units \r\nBlock F – 109 Units",4227964,100022980796,22838500,5930L000044821,HOMEBASE,E17 4EE,538468,190114,1,1
2,3311808,2866/21/IN,Construction of a two-storey three-bedroom dwelling house with a study in the roof space following\r\ndemolition of existing garage at land adjacent to 4 Thorpe Road.,4207777,2021-11-19,NaT,2021-11-22,NaT,NaT,Land adjacent to 4 Thorpe Road,4207777,100022591176,22881850,5930L000119976,,E17 4LA,538102,190265,1,1
3,3311809,2867/21/IN,Loft conversion at existing two storey dwelling.,4175244,2021-11-19,NaT,2022-04-25,NaT,2022-04-25,,4175244,100022573947,22862000,5930L000027374,,E17 7EB,536324,189124,1,1


In [15]:
##3. MATCH PLANNING AND BC

#run matching algorithm
planning_data.apply(lambda row: tascomi_utils.nearest_bc_apps(row, BC_data, matches_dict), axis=1)

#Create df from mapping dict
test_mapping = pd.DataFrame.from_dict(matches_dict)

#Drop applications further than 50m away
mask = test_mapping['bc_dist'] > 50
test_mapping = test_mapping.loc[~mask].copy()

In [16]:
test_mapping

,planning_id,bc_dist,bc_id
0,45979,0.0,3311793


In [17]:
#Merge planning and BC information to mapping df
test_mapping = test_mapping.merge(planning_data, on='planning_id', how='left')
test_mapping = test_mapping.merge(BC_data, left_on='bc_id', right_on='build_id', how='left')

In [18]:
print(f"""A possible Building Control application match has been found for planning application reference 
{planning_data['application_reference_number'][0]}

The application locations are {test_mapping['bc_dist'][0]}m apart

Planning application description:
{test_mapping['proposal'][0]}

Building Control application description (ref: {test_mapping['application_number'][0]}):
{test_mapping['description'][0]}""")

confirmation = str(input("Confirm match: Y for yes, N for no "))

A possible Building Control application match has been found for planning application reference 
202512

The application locations are 0.0m apart

Planning application description:
&lt;span style=&quot;font-size:11.0pt;font-family:&amp;quot;Arial&amp;quot;,sans-serif;
mso-fareast-font-family:Calibri;mso-fareast-theme-font:minor-latin;mso-ansi-language:
EN-GB;mso-fareast-language:EN-US;mso-bidi-language:AR-SA&quot;&gt;Demolition and
redevelopment of the site to provide a mixed use scheme comprising up to 583
residential units (Class C3 and including Private Rented Sector) in 8 buildings
ranging from 4 to 18 storeys, commercial uses (flexible retail, community and
leisure uses), new access from Fulbourne Road, car parking; provision of new
plant and renewable energy equipment; creation of servicing areas and provision
of associated services, including waste, refuse, cycle storage, and lighting;
and new routes and open spaces within the development and associated works and
operations. (Am

In [19]:
if confirmation == 'Y':
    cols = ['application_reference_number', 'received_date_x', 'decision_type_id', 'decision_issued_date', 'application_number', 'bc_id', 'received_date_y']
    match = test_mapping[cols].copy()

In [20]:
#populate planning approval and application dates
match = match.rename(columns={'received_date_x': 'planning_application'})

approval_types = ['2', '3', '4', '11', '13', '14', '15', '18']
match['planning_approval'] = pd.to_datetime(np.where(match['decision_type_id'].isin(approval_types), match['decision_issued_date'], pd.NaT))

In [21]:
#set up site visit column
match['wf_bc_site_visits'] = np.empty((len(match), 0)).tolist()

In [22]:
#join inspections
inspections = tascomi_utils.get_data('build', 'inspections', optional_param='inspected_date', optional_param_value='2022-01-13')
mask = match['bc_id'].isin(inspections['application_id'].tolist())
match.loc[mask] = match.loc[mask].apply(lambda x: tascomi_utils.join_inspections(x, inspections), axis=1)

In [23]:
#rename date column
bc_initial_notice = 'IN'

if bc_initial_notice in match['application_number']:
    match = match.rename(columns={'received_date_y': 'bc_initial_notice_submitted'})

else:
    match = match.rename(columns={'received_date_y': 'bc_application_submitted'})

In [24]:
match[['application_reference_number', 'planning_application', 'planning_approval', 'bc_application_submitted', 'wf_bc_site_visits']]

,application_reference_number,planning_application,planning_approval,bc_application_submitted,wf_bc_site_visits
0,202512,2020-08-19 00:00:00,2021-04-20,2021-11-19,[2022-01-13 00:00:00]


## Phase-level data

In [25]:
##2. EXTRACT EXACOM DATA
#Read in Exacom exports
CIL_sc = (
    pd.read_csv('../data/raw/Score_Card20220308.csv')
    .pipe(exacom_utils.clean_sc)
)

CIL_dv = (
    pd.read_csv('../data/raw/Demand_Values_Per_Authority_20220308.csv')
    .pipe(exacom_utils.clean_dv)
)

CIL_full = (
    exacom_utils.merge_reports(CIL_dv, CIL_sc)
    .pipe(exacom_utils.filter_columns)
)

In [26]:
CIL_dv = pd.read_csv('../data/raw/Demand_Values_Per_Authority_20220308.csv')

In [28]:
mask = CIL_full['planning_ref_number'] == '202512'
phase = CIL_full.loc[mask].copy()

In [29]:
phase

,planning_ref_number,Phase,Commencement Date,Form 2 Rec,LN Date,Form 6 Rec
439,202512,1,29-Jun-21,10-May-21,23-Jun-21,28-Jun-21
453,202512,2,25-Oct-21,09-Jul-21,02-Aug-21,21-Oct-21
454,202512,3,06-Dec-21,09-Jul-21,02-Aug-21,21-Oct-21
455,202512,4,10-Jan-22,09-Jul-21,02-Aug-21,21-Oct-21
456,202512,5,15-Nov-21,02-Sep-21,28-Sep-21,21-Oct-21
457,202512,6,25-Oct-21,02-Sep-21,20-Oct-21,21-Oct-21
